### To run the cbgt network, see first the technical_testbench to install all the required packages and start the server 

In [1]:
#Auto reloading 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload 

In [2]:
#Removing indentation issues:
!autopep8 --in-place --aggressive --aggressive init_params.py
!autopep8 --in-place --aggressive --aggressive popconstruct.py
!autopep8 --in-place --aggressive --aggressive generateepochs.py
!autopep8 --in-place --aggressive --aggressive qvalues.py

In [3]:
#Importing scripts:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct as popconstruct
import qvalues as qval
import generateepochs as gen
import pdb

2021-08-24 17:30:21,322	INFO worker.py:726 -- Connecting to existing Ray cluster at address: 192.168.202.150:6379


# Create pipelines

### Reward pipeline

In [4]:
rsg = cbgt.Pipeline() #rsg is short for 'reward schedule generator'


(rsg.volatile_pattern,
 rsg.cp_idx,
 rsg.cp_indicator,
 rsg.noisy_pattern,
 rsg.t1_epochs,
 rsg.t2_epochs,
) = rsg[gen.GenRewardSchedule](
        rsg.n_trials,
        rsg.volatility,
        rsg.conflict,
        rsg.reward_mu,
        rsg.reward_std,
    ).shape(6)

### Network pipeline

In [5]:
pl = cbgt.Pipeline()

In [6]:
#Adding rsg pipeline to the network pipeline: 
pl.add(rsg) 

In [7]:
#to update the Q-values 
pl.trial_num = 0 #first row of Q-values df - initialization data 
pl.chosen_action = 2 #chosen action for the current trial

In [8]:
#Defining necessary codeblocks: 

#MODIFIERS: 

#init_params.py: to modify the neuronal default values

def codeblock_modifycelldefaults(self):
    self.celldefaults = par.helper_cellparams(self.params)
    
def codeblock_modifypopspecific(self):
    self.popspecific = par.helper_popspecific(self.pops)
    
def codeblock_modifyreceptordefaults(self):
    self.receptordefaults = par.helper_receptor(self.receps)

def codeblock_modifybasestim(self):
    self.basestim = par.helper_basestim(self.base)
    
def codeblock_modifydpmndefaults(self):
    self.dpmndefaults = par.helper_dpmn(self.dpmns)
    
def codeblock_modifyd1defaults(self):
    self.d1defaults = par.helper_d1(self.d1)
    
def codeblock_modifyd2defaults(self):
    self.d2defaults = par.helper_d2(self.d2)

def codeblock_modifyactionchannels(self): 
    self.actionchannels = par.helper_actionchannels(self.channels)
    
#popconstruct.py: to modify population parameters 

def codeblock_popconstruct(self):
    self.popdata = popconstruct.helper_popconstruct(self.actionchannels, self.popspecific, self.celldefaults, self.receptordefaults, self.basestim, self.dpmndefaults, self.d1defaults, self.d2defaults)
    
def codeblock_poppathways(self): 
    self.pathways = popconstruct.helper_poppathways(self.popdata, self.newpathways) 

#init_params.py: Q-values initialization and update

def codeblock_init_Q_support_params(self):
    self.Q_support_params = qval.helper_init_Q_support_params()

def codeblock_update_Q_support_params(self,reward_val, chosen_action):
    self.Q_support_params = qval.helper_update_Q_support_params(self.Q_support_params,self.reward_val,pl.chosen_action)
    
def codeblock_init_Q_df(self):
    self.Q_df = qval.helper_init_Q_df(self.channels)
    
def codeblock_update_Q_df(self):
    self.Q_df, self.Q_support_params, self.dpmndefaults = qval.helper_update_Q_df(self.Q_df, self.Q_support_params, self.dpmndefaults,pl.trial_num)

In [9]:
#Defining necessary function modules: 

#init_params.py: default neuronal values 
pl.celldefaults = pl[par.helper_cellparams]()
pl.popspecific = pl[par.helper_popspecific]()
pl.receptordefaults = pl[par.helper_receptor]()
pl.basestim = pl[par.helper_basestim]()
pl.dpmndefaults = pl[par.helper_dpmn]()
pl.d1defaults = pl[par.helper_d1]()
pl.d2defaults = pl[par.helper_d2]()
#pl.actionchannels = pl[par.helper_actionchannels]()

#popconstruct.py: default population parameters 
pl.popdata = pl[popconstruct.helper_popconstruct](pl.actionchannels, pl.popspecific, pl.celldefaults, pl.receptordefaults, pl.basestim, pl.dpmndefaults, pl.d1defaults, pl.d2defaults)
pl.pathways = pl[popconstruct.helper_poppathways](pl.popdata)

#popconstruct.py: to create connectivity grids
pl.connectivity_AMPA, pl.meaneff_AMPA, pl.plastic_AMPA = pl[popconstruct.helper_connectivity]('AMPA', pl.popdata, pl.pathways).shape(3)
pl.connectivity_GABA, pl.meaneff_GABA, pl.plastic_GABA = pl[popconstruct.helper_connectivity]('GABA', pl.popdata, pl.pathways).shape(3)
pl.connectivity_NMDA, pl.meaneff_NMDA, pl.plastic_NMDA = pl[popconstruct.helper_connectivity]('NMDA', pl.popdata, pl.pathways).shape(3)

In [10]:
#Adding codeblocks to the newtork pipeline: 
pl.add(codeblock_modifycelldefaults)
pl.add(codeblock_modifypopspecific)
pl.add(codeblock_modifyreceptordefaults)
pl.add(codeblock_modifybasestim)
pl.add(codeblock_modifydpmndefaults)
pl.add(codeblock_modifyd1defaults)
pl.add(codeblock_modifyd2defaults)
pl.add(codeblock_modifyactionchannels)
pl.add(codeblock_popconstruct)
pl.add(codeblock_poppathways)

### Q-values pipeline

In [11]:
#Creating a new pipeline for Q-values: 
q_val_pipe = cbgt.Pipeline()

In [12]:
#Defining necessary function modules: 

#qvalues.py
q_val_pipe.Q_support_params = q_val_pipe[qval.helper_init_Q_support_params]()
rsg.reward_val = q_val_pipe[qval.get_reward_value](rsg.t1_epochs,rsg.t2_epochs,pl.chosen_action,pl.trial_num) 
q_val_pipe.Q_support_params = q_val_pipe[qval.helper_update_Q_support_params](q_val_pipe.Q_support_params,rsg.reward_val,pl.chosen_action)
q_val_pipe.Q_df = q_val_pipe[qval.helper_init_Q_df](pl.actionchannels)
(q_val_pipe.Q_df, q_val_pipe.Q_support_params, pl.dpmndefaults) = q_val_pipe[qval.helper_update_Q_df](q_val_pipe.Q_df,q_val_pipe.Q_support_params,pl.dpmndefaults,pl.trial_num).shape(3)

In [13]:
#Adding the q_val_pipe to the main network pipeline pl:
pl.add(q_val_pipe)

# Running the pipeline

### 1. Default values

In [14]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 
environment = {
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : None, #action channels related parameters (init_params.py)
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df':None, #initialized Q-values df (qvalues.py)  
    'n_trials': 5, #number of trials (generateepochs.py)
    'volatility': 2, #frequency of changepoints (generateepochs.py)
    'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [15]:
results.keys()

dict_keys(['params', 'pops', 'receps', 'base', 'dpmns', 'd1', 'd2', 'channels', 'actionchannels', 'newpathways', 'Q_support_params', 'Q_df', 'n_trials', 'volatility', 'conflict', 'reward_mu', 'reward_std', 'volatile_pattern', 'cp_idx', 'cp_indicator', 'noisy_pattern', 't1_epochs', 't2_epochs', 'trial_num', 'chosen_action', 'celldefaults', 'popspecific', 'receptordefaults', 'basestim', 'dpmndefaults', 'd1defaults', 'd2defaults', 'popdata', 'pathways', 'connectivity_AMPA', 'meaneff_AMPA', 'plastic_AMPA', 'connectivity_GABA', 'meaneff_GABA', 'plastic_GABA', 'connectivity_NMDA', 'meaneff_NMDA', 'plastic_NMDA', 'reward_val'])

In [16]:
results['celldefaults']

,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,Tau_ca,Eff_ca,...,g_adr_max,Vadr_h,Vadr_s,ADRRevPot,g_k_max,Vk_h,Vk_s,tau_k_max,n_k,h
0,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,80`,0.0`,...,0`,-100`,10`,-90`,0`,-34`,6.5`,8`,0`,1`


In [17]:
results['popdata']

,name,action,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,...,dpmn_DAp,dpmn_APRE,dpmn_APOST,dpmn_XPRE,dpmn_XPOST,dpmn_type,dpmn_alphaw,dpmn_a,dpmn_b,dpmn_c
0,GPi`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
1,GPi`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
2,STNE`,1`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
3,STNE`,2`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
4,GPeP`,1`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
5,GPeP`,2`,750`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
6,D1STR`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,1`,18.333333333333332`,1.0`,0.1`,0.05`
7,D1STR`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,1`,18.333333333333332`,1.0`,0.1`,0.05`
8,D2STR`,1`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,2`,-15.0`,0.5`,0.005`,0.05`
9,D2STR`,2`,75`,0.5`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,2`,-15.0`,0.5`,0.005`,0.05`


In [18]:
results['dpmndefaults']

,dpmn_tauDOP,dpmn_alpha,dpmn_DAt,dpmn_taum,dpmn_dPRE,dpmn_dPOST,dpmn_tauE,dpmn_tauPRE,dpmn_tauPOST,dpmn_wmax,dpmn_w,dpmn_Q1,dpmn_Q2,dpmn_m,dpmn_E,dpmn_DAp,dpmn_APRE,dpmn_APOST,dpmn_XPRE,dpmn_XPOST
0,2`,0.3`,0.0`,1e+100`,0.8`,0.04`,15`,15`,6`,0.3`,0.1286`,0.0`,0.0`,1.0`,0.0`,0.465156,0.0`,0.0`,0.0`,0.0`


### 2. Modifying a subset of values

In [19]:
environment = {
    'params': pd.DataFrame([[4, 1]], columns=['N', 'C']), 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), 
    'newpathways' : None, 
    'Q_support_params': None,
    'Q_df':None,
    'n_trials': 5,
    'volatility': 2,
    'conflict': 0.7,
    'reward_mu': 1,
    'reward_std': 0.1,
}

results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [20]:
results['celldefaults']

,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,Tau_ca,Eff_ca,...,g_adr_max,Vadr_h,Vadr_s,ADRRevPot,g_k_max,Vk_h,Vk_s,tau_k_max,n_k,h
0,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,80`,0.0`,...,0`,-100`,10`,-90`,0`,-34`,6.5`,8`,0`,1`


In [21]:
results['popdata']

,name,action,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,...,dpmn_DAp,dpmn_APRE,dpmn_APOST,dpmn_XPRE,dpmn_XPOST,dpmn_type,dpmn_alphaw,dpmn_a,dpmn_b,dpmn_c
0,GPi`,1`,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
1,GPi`,2`,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
2,STNE`,1`,750`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
3,STNE`,2`,750`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
4,GPeP`,1`,750`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
5,GPeP`,2`,750`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
6,D1STR`,1`,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,1`,18.333333333333332`,1.0`,0.1`,0.05`
7,D1STR`,2`,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,1`,18.333333333333332`,1.0`,0.1`,0.05`
8,D2STR`,1`,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,2`,-15.0`,0.5`,0.005`,0.05`
9,D2STR`,2`,4`,1`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,2`,-15.0`,0.5`,0.005`,0.05`


### 3. Modifying more values

In [25]:
environment = {
    'params': pd.DataFrame([[8, 30]], columns=['N', 'C']),
    'pops' : {'FSI': {'Taum': [60]}}, 
    'receps' : pd.DataFrame([[100, 100]], columns=['Tau_AMPA', 'Tau_GABA']), 
    'base' : {'FSI': {'FreqExt_AMPA': [100]}}, 
    'dpmns' : pd.DataFrame([[400, 100]], columns=['dpmn_wmax', 'dpmn_tauDOP']),
    'd1' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'd2' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'channels' : None,
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), 
    'newpathways' : pd.DataFrame([['LIP', 'D1STR', 'AMPA', 'syn', 700, 0.027]], columns=['src', 'dest', 'receptor', 'type', 'con', 'eff']), 
    'Q_support_params': None,
    'Q_df':None,
    'n_trials': 5,
    'volatility': 2,
    'conflict': 0.7,
    'reward_mu': 1,
    'reward_std': 0.1,
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [26]:
results['celldefaults']

,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,Tau_ca,Eff_ca,...,g_adr_max,Vadr_h,Vadr_s,ADRRevPot,g_k_max,Vk_h,Vk_s,tau_k_max,n_k,h
0,8`,30`,20`,-70`,-55`,-50`,-85`,0.5`,80`,0.0`,...,0`,-100`,10`,-90`,0`,-34`,6.5`,8`,0`,1`


In [27]:
results['popdata']

,name,action,N,C,Taum,RestPot,ResetPot,Threshold,RestPot_ca,Alpha_ca,...,dpmn_DAp,dpmn_APRE,dpmn_APOST,dpmn_XPRE,dpmn_XPOST,dpmn_type,dpmn_alphaw,dpmn_a,dpmn_b,dpmn_c
0,GPi`,1`,8`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
1,GPi`,2`,8`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
2,STNE`,1`,750`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
3,STNE`,2`,750`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
4,GPeP`,1`,750`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
5,GPeP`,2`,750`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`,nan`
6,D1STR`,1`,8`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,3`,18.333333333333332`,1.0`,0.1`,50`
7,D1STR`,2`,8`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,3`,18.333333333333332`,1.0`,0.1`,50`
8,D2STR`,1`,8`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,3`,-15.0`,0.5`,0.005`,50`
9,D2STR`,2`,8`,30`,20`,-70`,-55`,-50`,-85`,0.5`,...,0.0`,0.0`,0.0`,0.0`,0.0`,3`,-15.0`,0.5`,0.005`,50`
